# Abstract Translator Test Case (in style of api call)

In [8]:
import sbol2

from sbol2build.abstract_translator import (
    translate_abstract_to_plasmids,
    extract_toplevel_definition,
)

use `pip install -e .` and restart kernel to reload latest version of local package

In [9]:
plasmid_collection = sbol2.Document()
sbh = sbol2.PartShop("https://synbiohub.org")
sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/CIDARMoCloPlasmidsKit_collection/1/b7fdc21c6601a61d3166073a9e50f2c3843e1df5/share",
    plasmid_collection,
)
sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloParts/CIDARMoCloParts_collection/1/038feff5f9c6c9b35cc95586ed6b14ec9bd3fea5/share",
    plasmid_collection,
)

abstract_design_doc = sbol2.Document()
abstract_design_doc.read("tests/test_files/moclo_parts_circuit.xml")

backbone_doc = sbol2.Document()
sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/DVK_AE/1/647c5b2458567dcce6b0a37178d352b8ffa9a7fe/share",
    backbone_doc,
)

print(extract_toplevel_definition(backbone_doc).displayId)

mocloplasmid_list = translate_abstract_to_plasmids(
    abstract_design_doc, plasmid_collection, backbone_doc
)

print(mocloplasmid_list)

DVK_AE
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_AB/1 with ['Fusion_Site_A', 'Fusion_Site_B']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_GB/1 with ['Fusion_Site_B', 'Fusion_Site_G']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_FB/1 with ['Fusion_Site_F', 'Fusion_Site_B']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_EB/1 with ['Fusion_Site_E', 'Fusion_Site_B']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/B0034/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pB0034_BC/1 with ['Fusion_Site_B', 'Fusion_Site_C']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/E0030_yfp/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pE0030_CD/1 w

## Run through OG sbol2build

In [10]:
from sbol2build.abstract_translator import extract_design_parts


def copy_sequences(component_definition, target_doc, collection_doc):
    """Copy all sequences referenced by a ComponentDefinition into target_doc."""
    subdefinitions = extract_design_parts(component_definition, collection_doc)

    for seq_uri in component_definition.sequences:
        seq_obj = component_definition.doc.find(seq_uri)
        if seq_obj is not None:
            seq_obj.copy(target_doc)

    for subdefinition in subdefinitions:
        print(subdefinition.displayId)
        subdefinition.copy(target_doc)
        for seq_uri in subdefinition.sequences:
            seq_obj = component_definition.doc.find(seq_uri)
            if seq_obj is not None:
                seq_obj.copy(target_doc)

In [11]:
from sbol2build import (
    golden_gate_assembly_plan,
    part_digestion,
    rebase_restriction_enzyme,
)

part_documents = []
restriction_enzyme = rebase_restriction_enzyme("BsaI")
assembly_plan = sbol2.ModuleDefinition()

for mocloPlasmid in mocloplasmid_list:
    new_doc = sbol2.Document()
    mocloPlasmid.definition.copy(new_doc)
    copy_sequences(mocloPlasmid.definition, new_doc, plasmid_collection)
    part_documents.append(new_doc)

    # Run digestion
    extracted, assembly_md = part_digestion(
        mocloPlasmid.definition, [restriction_enzyme], assembly_plan, new_doc
    )
    print(assembly_md, [(c.displayId, s.elements[:8]) for c, s in extracted])

assembly_doc = sbol2.Document()
assembly_obj = golden_gate_assembly_plan(
    "composite_2", part_documents, backbone_doc, "BsaI", assembly_doc
)

Fusion_Site_A
J23100
Fusion_Site_B
dva_backbone_core
https://SBOL2Build.org/example/1 [('pJ23100_AB_extracted_part', 'GGAGTTGA'), ('pJ23100_AB_five_prime_oh', 'GGAG'), ('Fusion_Site_A', 'GGAG'), ('J23100', 'TTGACGGC'), ('pJ23100_AB_three_prime_oh', 'TACT'), ('Fusion_Site_B', 'TACT')]
Fusion_Site_B
B0034
Fusion_Site_C
dva_backbone_core
https://SBOL2Build.org/example/1 [('pB0034_BC_extracted_part', 'TACTAGAG'), ('pB0034_BC_five_prime_oh', 'TACT'), ('Fusion_Site_B', 'TACT'), ('B0034', 'AGAGAAAG'), ('pB0034_BC_three_prime_oh', 'AATG'), ('Fusion_Site_C', 'AATG')]
Fusion_Site_C
E0030_yfp
Fusion_Site_D
dva_backbone_core
https://SBOL2Build.org/example/1 [('pE0030_CD_extracted_part', 'AATGATGG'), ('pE0030_CD_five_prime_oh', 'AATG'), ('Fusion_Site_C', 'AATG'), ('E0030_yfp', 'ATGGTGAG'), ('pE0030_CD_three_prime_oh', 'AGGT'), ('Fusion_Site_D', 'AGGT')]
Fusion_Site_D
B0015
Fusion_Site_E
dva_backbone_core
https://SBOL2Build.org/example/1 [('pB0015_DE_extracted_part', 'AGGTCCAG'), ('pB0015_DE_five_pr

In [12]:
assembly_obj.run()
assembly_doc.write("abstract_assembly_plan.xml")

adding: pJ23100_AB_extracted_part
adding: pJ23100_AB_five_prime_oh
adding: Fusion_Site_A
adding: J23100
adding: pJ23100_AB_three_prime_oh
adding: Fusion_Site_B
adding: pB0034_BC_extracted_part
adding: pB0034_BC_five_prime_oh
adding: Fusion_Site_B
adding: B0034
adding: pB0034_BC_three_prime_oh
adding: Fusion_Site_C
adding: pE0030_CD_extracted_part
adding: pE0030_CD_five_prime_oh
adding: Fusion_Site_C
adding: E0030_yfp
adding: pE0030_CD_three_prime_oh
adding: Fusion_Site_D
adding: pB0015_DE_extracted_part
adding: pB0015_DE_five_prime_oh
adding: Fusion_Site_D
adding: B0015
adding: pB0015_DE_three_prime_oh
adding: Fusion_Site_E
adding: DVK_AE_extracted_backbone
adding: DVK_AE_three_prime_oh
adding: Fusion_Site_A
adding: DVK_AE_five_prime_oh
adding: Fusion_Site_E
adding: dvk_backbone_core
adding: composite_1


'Valid.'

In [13]:
rbs_2 = sbol2.Document()
sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pB0032_BC/1/620c77db9353a6d4a2bd5fe4fdbe080e6bc379c2/share",
    rbs_2,
)

part_documents.append(rbs_2)

print(part_documents)

comb_assembly_doc = sbol2.Document()
comb_assembly_obj = golden_gate_assembly_plan(
    "combinatorial_assembly_plan",
    part_documents,
    backbone_doc,
    "BsaI",
    comb_assembly_doc,
)

comb_assembly_obj.run()
comb_assembly_doc.write("comb_assembly_plan.xml")

[<sbol2.document.Document object at 0x10a59d810>, <sbol2.document.Document object at 0x10a59e990>, <sbol2.document.Document object at 0x10a59cf50>, <sbol2.document.Document object at 0x10a59e5d0>, <sbol2.document.Document object at 0x10a59efd0>]
adding: pJ23100_AB_extracted_part
adding: pJ23100_AB_five_prime_oh
adding: Fusion_Site_A
adding: J23100
adding: pJ23100_AB_three_prime_oh
adding: Fusion_Site_B
adding: pB0034_BC_extracted_part
adding: pB0034_BC_five_prime_oh
adding: Fusion_Site_B
adding: B0034
adding: pB0034_BC_three_prime_oh
adding: Fusion_Site_C
adding: pE0030_CD_extracted_part
adding: pE0030_CD_five_prime_oh
adding: Fusion_Site_C
adding: E0030_yfp
adding: pE0030_CD_three_prime_oh
adding: Fusion_Site_D
adding: pB0015_DE_extracted_part
adding: pB0015_DE_five_prime_oh
adding: Fusion_Site_D
adding: B0015
adding: pB0015_DE_three_prime_oh
adding: Fusion_Site_E
adding: pB0032_BC_extracted_part
adding: B0032
adding: pB0032_BC_five_prime_oh
adding: Fusion_Site_B
adding: pB0032_BC_thr

'Valid.'